# Network randomization

Here, permutation testing is used to asess significant connectivity between LSNs. Association between two LSNs is considered significant when mean connection strenght between them is higher that 95% values from null distribution. 

Null distribution of between LSN connectivity is calculated using `n_nulls` random networks. Each random network have same total strenght and degree distribution (but not strenght distribution) as original network. Algorithm to rewire network edges `randmio_und` comes from BCT. Each edge is rewired approximately `n_rewirings` times. 

In [ ]:
import json
import os
import pathlib
from os.path import join

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from bct.algorithms.reference import randmio_und, null_model_und_sign
from bct.utils import BCTParamError
from dn_utils.plotting import plot_matrix
from dn_utils.networks import networks_mean

%matplotlib inline

In [ ]:
path_root = os.environ.get("DECIDENET_PATH")

path_derivatives = join(path_root, "data/main_fmri_study/derivatives")
path_sourcedata = join(path_root, "data/main_fmri_study/sourcedata") 

path_beh = join(path_sourcedata, "behavioral")
path_bsc = join(path_derivatives, "bsc")
path_nistats = join(path_derivatives, "nistats")
path_parcellations = join(path_derivatives, "parcellations")

path_corrmats = join(path_bsc, "corrmats")

In [ ]:
# Input networks
atlas = "combined_roi"
alpha_fdr = 1e-10

# Randomization options
n_nulls = 5
n_rewirings = 1

# Create output paths
alpha_fdr_str = str(alpha_fdr).replace("-", "")
path_out = join(path_corrmats, atlas, f"fdrthr_{alpha_fdr_str}")
path_nulls = join(path_out, "nulls")
pathlib.Path(path_nulls).mkdir(exist_ok=True, parents=True)

In [ ]:
# Load correlation matrices and metadata
corrmats_aggregated = np.load(join(path_corrmats, atlas, 
                                   "corrmats_aggregated.npy"))
with open(join(path_corrmats, atlas, "corrmats_aggregated.json"), "r") as f:
    meta = json.loads(f.read())

# Load subject exclusion
df_exclusion = pd.read_csv(join(path_nistats, "exclusion/exclusion.csv"), 
                           index_col=0)
ok_index = df_exclusion["ok_all"]    
    
# Load ROI information
df_roi = pd.read_csv(join(path_corrmats, atlas, "roi_table_filtered.csv"))
netnames = df_roi["netName"].unique()

n_subjects = len(meta["dim1"])
n_conditions = len(meta["dim2"])
n_perr_sign = len(meta["dim3"])
n_rois = len(df_roi)
n_nets = len(netnames)

# Load pvalue mask
pvalues_mask = np.load(join(path_out, f"pvalues_mask.npy"))

In [ ]:
# Thresholding
corrmats_aggregated_thr = corrmats_aggregated * pvalues_mask
corrmats_aggregated_thr = corrmats_aggregated_thr * (corrmats_aggregated_thr > 0)

for con_idx, con in enumerate(meta["dim2"]):
    for perr_sign_idx, perr_sign in enumerate(meta["dim3"]):
        for sub_idx, sub in enumerate(meta["dim1"]): 
            print(f"Randomizing {sub} | {con} | {perr_sign}")

            # Extract correlation matrix & clean diagonal
            corrmat = corrmats_aggregated_thr[sub_idx, con_idx, perr_sign_idx]
            corrmat[np.diag_indices_from(corrmat)] = 0

            meanmat_nulls = np.zeros((n_nulls, n_nets, n_nets))
            
            for rep in tqdm(range(n_nulls)):

                # Randomize network
                try:
#                     corrmat_null, _  = null_model_und_sign(corrmat, n_rewirings)
                    corrmat_null, _  = randmio_und(corrmat, n_rewirings)
                except BCTParamError:
                    # In case of non-symmetrical matrix from failed acquisition
                    corrmat_null = np.zeros((n_rois, n_rois))

                # Calculate lsn mean connectivity & store
                _, meanmat_nulls[rep] = networks_mean(corrmat_null, 
                                                      df_roi["netName"])

            fname = f"sub-{sub}_task-prl{con}_perrsign-{perr_sign[-3:]}_nullmeanmats.npy"
            np.save(join(path_nulls, fname), meanmat_nulls)